# Writing Blog Posts with DSPy Programs


Sources:
- Video: https://www.youtube.com/watch?v=ickqCzFxWj0
- Repo: https://github.com/weaviate/recipes/blob/main/integrations/dspy/2.Writing-Blog-Posts-with-DSPy.ipynb 


In this tutorial, we will generate blog posts from questions.

Using the Weaviate blog posts as background information, we will generate 8 paragraph blog posts from questions such as:

"What is ref2vec and how can I use it to build e-commerce applications with Weaviate?"

To run this:

1. Excecute the cells under "1. Setup"
2. Start Docker Compose file (see end of 1. Setup-section)
3. (If not already done): Excecute cells under "2. Import blog into Weaviate"
4. Start writing by executing cells under "Excecute cells under 3. Writing Blog Posts with DSPy Programs" 


# 1. Setup

In [ ]:
!pip install dspy-ai weaviate-client==4.5.4

In [2]:
import os
import json
from getpass import getpass
import psutil

In [ ]:
IN_NOTEBOOK = any(["jupyter-notebook" in i for i in psutil.Process().parent().cmdline()])
if IN_NOTEBOOK:
  CREDS = json.loads(getpass("Secrets (JSON string): "))
  os.environ['CREDS'] = json.dumps(CREDS)
  CREDS = json.loads(os.getenv('CREDS'))

In [4]:
os.environ["OPENAI_API_KEY"] = CREDS['OpenAI']['v2']['credential'] 

In [5]:
# create .env file with OpenAI key, needed for Docker
f= open('.env','w+')
f.write('OPENAI_APIKEY=' + CREDS['OpenAI']['v2']['credential'])
f.close()

In [6]:
if not os.path.isdir('./blog'):
    print('"blog" folder is missing. See documentation above!')
else:
    print('Setup ready. Start docker.')

Setup ready. Start docker.


Now run in terminal `docker compose up -d`

# 2. Import blog into Weaviate

### Weaviate Import


Source: https://github.com/weaviate/recipes/tree/main/integrations/dspy

This section is used to populate the `WeaviateBlogChunk` class.

1. Run `docker-compose up -d` with the docker script in the file to start Weaviate locally on localhost:8080

2. Make sure the `/blog` folder is in this directory (these are parsed from github.com/weaviate/weaviate-io -- feel free to drag and drop that folder in here to update the content).

3. Run this notebook and the 1182 blog chunks will be loaded into Weaviate (takes approx. 6-7 minutes)

In [ ]:
# Import Weaviate and Connect to Client
import weaviate
client = weaviate.Client("http://localhost:8080")

In [ ]:
# Create Schema
schema = {
   "classes": [
       {
           "class": "WeaviateBlogChunk",
           "description": "A snippet from a Weaviate blogpost.",
           "moduleConfig": {
               "text2vec-openai": {
                    "skip": False,
                    "vectorizeClassName": False,
                    "vectorizePropertyName": False
                },
                "generative-openai": {
                    "model": "gpt-3.5-turbo"
                }
           },
           "vectorIndexType": "hnsw",
           "vectorizer": "text2vec-openai",
           "properties": [
               {
                   "name": "content",
                   "dataType": ["text"],
                   "description": "The text content of the podcast clip",
                   "moduleConfig": {
                    "text2vec-transformers": {
                        "skip": False,
                        "vectorizePropertyName": False,
                        "vectorizeClassName": False
                    }
                   }
               },
               {
                "name": "author",
                "dataType": ["text"],
                "description": "The author of the blog post.",
                "moduleConfig": {
                    "text2vec-openai": {
                        "skip": True,
                        "vectorizePropertyName": False,
                        "vectorizeClassName": False
                    }
                }
               }
           ]
       }      
   ]
}
    
client.schema.create(schema)

In [ ]:
import os
import re

def chunk_list(lst, chunk_size):
    """Break a list into chunks of the specified size."""
    return [lst[i:i + chunk_size] for i in range(0, len(lst), chunk_size)]

def split_into_sentences(text):
    """Split text into sentences using regular expressions."""
    sentences = re.split(r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s', text)
    return [sentence.strip() for sentence in sentences if sentence.strip()]

def read_and_chunk_index_files(main_folder_path):
    """Read index.md files from subfolders, split into sentences, and chunk every 5 sentences."""
    blog_chunks = []
    for folder_name in os.listdir(main_folder_path):
        subfolder_path = os.path.join(main_folder_path, folder_name)
        if os.path.isdir(subfolder_path):
            index_file_path = os.path.join(subfolder_path, 'index.mdx')
            if os.path.isfile(index_file_path):
                with open(index_file_path, 'r', encoding='utf-8') as file:
                    content = file.read()
                    sentences = split_into_sentences(content)
                    sentence_chunks = chunk_list(sentences, 5)
                    sentence_chunks = [' '.join(chunk) for chunk in sentence_chunks]
                    blog_chunks.extend(sentence_chunks)
    return blog_chunks

# Example usage
main_folder_path = './blog'
blog_chunks = read_and_chunk_index_files(main_folder_path)


In [ ]:
len(blog_chunks)

In [ ]:
blog_chunks[0]

In [ ]:
client.batch.configure(
  # `batch_size` takes an `int` value to enable auto-batching
  # (`None` is used for manual batching)
  batch_size=100,
  # dynamically update the `batch_size` based on import speed
  dynamic=False,
  # `timeout_retries` takes an `int` value to retry on time outs
  timeout_retries=3,
  # checks for batch-item creation errors
  # this is the default in weaviate-client >= 3.6.0
  callback=weaviate.util.check_batch_result,
)

In [ ]:
from weaviate.util import get_valid_uuid
from uuid import uuid4
import time
start = time.time()
for idx, blog_chunk in enumerate(blog_chunks):
    data_properties = {
        "content": blog_chunk
    }
    id = get_valid_uuid(uuid4())
    with client.batch as batch:
        batch.add_data_object(
            data_properties,
            "WeaviateBlogChunk"
        )
    '''
    client.data_object.create(
        data_object = data_properties,
        class_name = "WeaviateBlogChunk",
        uuid=id
    )
    '''

print(f"Uploaded {idx} documents in {time.time() - start} seconds.")

# 3. Writing Blog Posts with DSPy Programs

### In this tutorial, we will generate blog posts from questions!

Using the Weaviate blog posts as background information, we will generate 8 paragraph blog posts from questions such as:

`What is ref2vec and how can I use it to build e-commerce applications with Weaviate?`

### We will do this with a 4-layer DSPy program

```python
class BlogPostWriter(dspy.Module):
    def __init__(self):
        self.question_to_blog_outline = dspy.ChainOfThought(Question2BlogOutline)
        self.topic_to_paragraph = dspy.ChainOfThought(Topic2Paragraph)
        self.proof_reader = dspy.ChainOfThought(ProofReader)
        self.title_generator = dspy.ChainOfThought(TitleGenerator)
    
    def forward(self, question):
        contexts = dspy.Retrieve(k=5)(question).passages
        contexts = "".join(contexts)
        raw_blog_outline = self.question_to_blog_outline(question=question, context=contexts).blog_outline
        blog_outline = raw_blog_outline.split(",") #Add type hint in expanded Signature
        blog = ""
        for topic in blog_outline:
            topic_contexts = dspy.Retrieve(k=5)(topic).passages
            topic_contexts = "".join(topic_contexts)
            blog += self.topic_to_paragraph(topic=topic, contexts=topic_contexts).paragraph
            blog += "\n \n"
        blog = self.proof_reader(blog_post=blog).proofread_blog_post
        title = self.title_generator(blog_outline=raw_blog_outline).title
        final_blog = f"{title} \n \n {blog}"
        return dspy.Prediction(blog=final_blog)
```

## Installation

In [7]:
# Setup DSPy
# Connect to Weaviate Retriever and configure LLM
import dspy
from dspy.retrieve.weaviate_rm import WeaviateRM
import weaviate

gpt4 = dspy.OpenAI(model="gpt-4", max_tokens=2000, model_type="chat")

weaviate_client = weaviate.Client("http://localhost:8080")
retriever_model = WeaviateRM("WeaviateBlogChunk", weaviate_client=weaviate_client)
dspy.settings.configure(lm=gpt4, rm=retriever_model)

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/weaviate/warnings.py:158: DeprecationWarning: Dep016: You are using the Weaviate v3 client, which is deprecated.
            Consider upgrading to the new and improved v4 client instead!
            See here for usage: https://weaviate.io/developers/weaviate/client-libraries/python
            
  warnings.warn(


### Steps to Generate a Blog Post
#### 1. Outline
#### 2. Topic to paragraph
#### 3. Review
#### 4. Title

## Question2BlogOutline Signature and Demo

In [8]:
class Question2BlogOutline(dspy.Signature):
    """
    Your task is to write a blog post that will help answer the given question. 
    Please use the contexts to evaluate the structure of the blog post.
    """
    
    question = dspy.InputField()
    contexts = dspy.InputField()
    blog_outline = dspy.OutputField(desc="A comma separated list of topics.")

In [9]:
toy_question = "How does Hybrid Search in Weaviate work?"
toy_context = dspy.Retrieve(k=5)(toy_question).passages
toy_context = "".join(toy_context)
dspy.ChainOfThought(Question2BlogOutline)(question=toy_question, contexts=toy_context).blog_outline

"Introduction to Hybrid Search, How Hybrid Search Works in Weaviate, Fusion Algorithms in Weaviate's Hybrid Search, Example of a Hybrid Search, Combining Results in Hybrid Search."

In [10]:
gpt4.inspect_history(n=1)





Your task is to write a blog post that will help answer the given question. 
    Please use the contexts to evaluate the structure of the blog post.

---

Follow the following format.

Question: ${question}

Contexts: ${contexts}

Reasoning: Let's think step by step in order to ${produce the blog_outline}. We ...

Blog Outline: A comma separated list of topics.

---

Question: How does Hybrid Search in Weaviate work?

Contexts: ### Further resources - [How-to: Hybrid search](/developers/weaviate/search/hybrid) - [API References: Hybrid search](/developers/weaviate/api/graphql/search-operators#hybrid)But do you know **how** hybrid search combines these results? And that Weaviate recently added a new algorithm for how this is done? In this post, we’ll dive into exactly the world of hybrid search to discuss how it works, how results are produced, the algorithms used, and more. So let’s get into it! :::info - Vector search and keyword search are also known as dense vector search and sp

## Topic to Paragraph

In [11]:
class Topic2Paragraph(dspy.Signature):
    """
    Your task is to write a paragraph that explains a topic based on the retrieved contexts.
    """
    
    topic = dspy.InputField(desc="A topic to write a paragraph about based on the information in the contexts.")
    contexts = dspy.InputField(desc="contains relevant information about the topic.")
    paragraph = dspy.OutputField()

In [12]:
toy_topic = "How does Hybrid Search in Weaviate work?"
toy_topic_context = dspy.Retrieve(k=5)(toy_topic).passages
toy_topic_context = "".join(toy_topic_context)
dspy.ChainOfThought(Topic2Paragraph)(topic=toy_topic, contexts=toy_topic_context).paragraph

'Hybrid search in Weaviate is a sophisticated technique that combines multiple search algorithms to enhance the accuracy and relevance of search results. It integrates the best features of both keyword-based search algorithms, also known as BM25 search in Weaviate, and vector search techniques. The hybrid search uses sparse and dense vectors to encapsulate the meaning and context of search queries and documents. There are two fusion algorithms available in Weaviate for this purpose: `rankedFusion` and `relativeScoreFusion`. In essence, a hybrid search performs a vector search to find the most similar objects to the vector of the query, and a keyword search that ranks results based on the frequency of the query terms. The results from both these searches are then combined to provide a more effective and accurate search experience.'

In [13]:
gpt4.inspect_history(n=1)





Your task is to write a paragraph that explains a topic based on the retrieved contexts.

---

Follow the following format.

Topic: A topic to write a paragraph about based on the information in the contexts.

Contexts: contains relevant information about the topic.

Reasoning: Let's think step by step in order to ${produce the paragraph}. We ...

Paragraph: ${paragraph}

---

Topic: How does Hybrid Search in Weaviate work?

Contexts: ### Further resources - [How-to: Hybrid search](/developers/weaviate/search/hybrid) - [API References: Hybrid search](/developers/weaviate/api/graphql/search-operators#hybrid)But do you know **how** hybrid search combines these results? And that Weaviate recently added a new algorithm for how this is done? In this post, we’ll dive into exactly the world of hybrid search to discuss how it works, how results are produced, the algorithms used, and more. So let’s get into it! :::info - Vector search and keyword search are also known as dense vector search

## Proof Reader Signature

In [14]:
class ProofReader(dspy.Signature):
    """
    Proofread a blog post and output a more well written version of the original post.
    """
    
    blog_post = dspy.InputField()
    proofread_blog_post = dspy.OutputField()    

## Title Generator

In [15]:
class TitleGenerator(dspy.Signature):
    """
    Write a title for a blog post given a description of the topics the blog covers as input.
    """
    
    blog_outline = dspy.InputField()
    title = dspy.OutputField()

## Piecing it all together

In [16]:
class BlogPostWriter(dspy.Module):
    def __init__(self):
        self.question_to_blog_outline = dspy.ChainOfThought(Question2BlogOutline)
        self.topic_to_paragraph = dspy.ChainOfThought(Topic2Paragraph)
        self.proof_reader = dspy.ChainOfThought(ProofReader)
        self.title_generator = dspy.ChainOfThought(TitleGenerator)
    
    def forward(self, question):
        contexts = dspy.Retrieve(k=5)(question).passages
        contexts = "".join(contexts)
        raw_blog_outline = self.question_to_blog_outline(question=question, contexts=contexts).blog_outline
        blog_outline = raw_blog_outline.split(",") #Add type hint in expanded Signature
        blog = ""
        for topic in blog_outline:
            topic_contexts = dspy.Retrieve(k=5)(topic).passages
            topic_contexts = "".join(topic_contexts)
            blog += self.topic_to_paragraph(topic=topic, contexts=topic_contexts).paragraph
            blog += "\n \n"
        blog = self.proof_reader(blog_post=blog).proofread_blog_post
        title = self.title_generator(blog_outline=raw_blog_outline).title
        final_blog = f"{title} \n \n {blog}"
        return dspy.Prediction(blog=final_blog)

In [17]:
hybrid_search_topic = "How does Hybrid Search in Weaviate work?"
hybrid_search_blog = BlogPostWriter()(hybrid_search_topic)
print(hybrid_search_blog.blog)

"Understanding Weaviate's Hybrid Search: Functioning, Fusion Algorithms, and Result Combination" 
 
 Hybrid search is a powerful technique that enhances the accuracy and relevance of search results by merging the best features of keyword-based search algorithms and vector search techniques. This method is employed in Weaviate, a platform that introduced this feature in its 1.17 version. 

Hybrid search uses both dense and sparse vectors to represent the meaning and context of search queries and documents. Dense vectors excel at understanding the context of the query, while sparse vectors are adept at keyword matches. This combination allows for more accurate and contextually relevant search results. 

Weaviate offers two fusion algorithms for hybrid search: `rankedFusion` and `relativeScoreFusion`. The `rankedFusion` algorithm is the default, but the newer `relativeScoreFusion` is considered a better choice for most users. These algorithms work by preparing the scores from each search 

In [18]:
ecommerce_topic = "e-Commerce reimagined with Weaviate"
ecommerce_blog = BlogPostWriter()(ecommerce_topic)
print(ecommerce_blog.blog)

"Leveraging Weaviate for e-Commerce: A Case Study of Moonsift" 
 
 Weaviate is an open-source, cloud-native, modular, real-time vector database designed to enhance machine learning models. It's not just a data storage system, but also a knowledge representation tool that enables semantic search and automatic classification. Weaviate stores data along with its automatically derived context, including linguistic information and relations to other concepts, resulting in a knowledge network or data graph. The project is community-driven, offering resources such as workshops, a YouTube channel, a blog, and a quickstart guide for those interested in learning more or contributing.

Weaviate combines the speed and capabilities of Approximate Nearest Neighbors (ANN) algorithms with the features of a traditional database, such as backups, real-time queries, persistence, and replication. It can be accessed through GraphQL, REST, and client libraries in multiple programming languages. It's particu

## BlogRater Program

In [19]:
class BlogRater(dspy.Signature):
    """
    Rate a blog post on a scale of 1 to 5 on how well-written it is.
    """
    
    blog = dspy.InputField(desc="a blog post")
    rating = dspy.OutputField(desc="a quality rating on a scale of 1 to 5. IMPORTANT!! ONLY OUTPUT THE RATING AS A FLOAT VALUE AND NOTHING ELSE!")

class MetricProgram(dspy.Module):
    def __init__(self):
        self.rater = dspy.ChainOfThought(BlogRater)
    
    def forward(self, gold, pred, trace=None):
        blog = pred.blog
        gold = gold.question
        return float(self.rater(blog=blog).rating)

def MetricWrapper(gold, pred, trace=None):
    return MetricProgram()(gold=gold, pred=pred)

In [20]:
MetricProgram()(gold=dspy.Example(question=hybrid_search_topic).with_inputs("question"), pred=hybrid_search_blog)

4.8

In [21]:
MetricProgram()(gold=dspy.Example(question=ecommerce_topic).with_inputs("question"), pred=ecommerce_blog)

4.7

In [22]:
gpt4.inspect_history(n=4)





Proofread a blog post and output a more well written version of the original post.

---

Follow the following format.

Blog Post: ${blog_post}
Reasoning: Let's think step by step in order to ${produce the proofread_blog_post}. We ...
Proofread Blog Post: ${proofread_blog_post}

---

Blog Post: Weaviate is an open-source, cloud-native, modular, real-time vector database designed to scale machine learning models. It is not just a data storage system, but also a knowledge representation tool that enables semantic search and automatic classification. Weaviate stores data along with its automatically derived context, which includes linguistic information and relations to other concepts, resulting in a network of knowledge or a data graph. The project is community-driven, with resources such as workshops, a YouTube channel, a blog, and a quickstart guide available for those interested in learning more or contributing to the project. Weaviate is a cloud-native, modular, real-time vector d

In [23]:
trainset = [
    dspy.Example(question="What is multi-tenancy?").with_inputs("question"),
    dspy.Example(question="What is product quantization?").with_inputs("question"),
    dspy.Example(question="How does asynchronous indexing work in Weaviate?").with_inputs("question"),
    dspy.Example(question="What are cross encoders?").with_inputs("question")
]
testset = [
    dspy.Example(question="What is a vector database?").with_inputs("question"),
    dspy.Example(question="What is HNSW?").with_inputs("question"),    
    dspy.Example(question="What can I build with Cohere and Weaviate?").with_inputs("question"),
    dspy.Example(question="What is a race condition in database systems?").with_inputs("question"),
    dspy.Example(question="How do inverted indexes work?").with_inputs("question")
]

##  Evaluate Uncompiled

In [24]:
from dspy.evaluate.evaluate import Evaluate

evaluate = Evaluate(devset=testset, num_threads=1, display_progress=True, display_table=5)

evaluate(BlogPostWriter(), metric=MetricWrapper)

Average Metric: 23.0 / 5  (460.0): 100%|██████████| 5/5 [06:01<00:00, 72.36s/it]


Average Metric: 23.0 / 5  (460.0%)


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/pyparsing.py:108: DeprecationWarning: module 'sre_constants' is deprecated
  import sre_constants


,question,blog,MetricWrapper
0,What is a vector database?,"""A Comprehensive Guide to Vector Databases: Definition, Comparison, Use Cases, and More"" A vector database is a modern type of database that stores data indexed...",5.0
1,What is HNSW?,"""Understanding HNSW: Features, Comparison with Vamana, and Limitations"" Hierarchical Navigable Small World (HNSW) is a graph-based method for finding nearest neighbors in a dataset. It...",4.5
2,What can I build with Cohere and Weaviate?,"""Harnessing the Power of Cohere and Weaviate: A Comprehensive Guide to Integration and Project Building"" Cohere is an AI platform specializing in Natural Language Processing...",4.5
3,What is a race condition in database systems?,"""Solving Race Conditions in Database Systems with the Lock Striping Pattern: A Comprehensive Guide"" A race condition is a common issue in database systems, particularly...",4.5
4,How do inverted indexes work?,"""In-Depth Understanding of Inverted Indexes: Roles, Organization, Metrics, and Comparison with Bitmap Indexes"" An inverted index is a key component in data storage and retrieval...",4.5


460.0

## Compile

In [25]:
from dspy.teleprompt import BootstrapFewShot

teleprompter = BootstrapFewShot(metric=MetricWrapper, max_bootstrapped_demos=1,max_rounds=1)

# also common to init here, e.g. Rag()
compiled_blog_writer = teleprompter.compile(BlogPostWriter(), trainset=trainset)

 25%|███████████▎                                 | 1/4 [01:16<03:48, 76.10s/it]

Bootstrapped 1 full traces after 2 examples in round 0.


## Save Compiled Program

In [27]:
# Save the program
compiled_blog_writer.save("compiled_blog_writer.json")

## Load Compiled Program

In [34]:
# Load the program
compiled_blog_writer = BlogPostWriter()
compiled_blog_writer.load("compiled_blog_writer.json")

## Quick Test

In [31]:
print(compiled_blog_writer("What is a Self-Driving Database?").blog)

"The Evolution and Future of Database Management: Unpacking the Self-Driving Database" 

---

Blog Outline: Introduction to Veganism, Health Benefits of a Vegan Diet, Environmental Impact of Veganism, Common Myths about Veganism, Tips for Transitioning to a Vegan Lifestyle.
Reasoning: Let's think step by step in order to produce the title. The blog covers various aspects of veganism, including its health benefits, environmental impact, and common myths. It also provides tips for transitioning to a vegan lifestyle. The title should reflect these points and indicate that the blog will provide a comprehensive guide to veganism.
Title: "Embracing Veganism: Health, Environment, Myths, and Transitioning Tips"

---

Blog Outline: Understanding Cryptocurrency, The Rise of Bitcoin, How Blockchain Technology Works, Investing in Cryptocurrency, Future Predictions for Cryptocurrency.
Reasoning: Let's think step by step in order to produce the title. The blog is about cryptocurrency, with a focus o

In [36]:
gpt4.inspect_history(n=10)





Your task is to write a paragraph that explains a topic based on the retrieved contexts.

---

Follow the following format.

Topic: A topic to write a paragraph about based on the information in the contexts.

Contexts: contains relevant information about the topic.

Reasoning: Let's think step by step in order to ${produce the paragraph}. We ...

Paragraph: ${paragraph}

---

Topic: Definition of Multi-Tenancy

Contexts: But if you have a multi-tenancy case, all you need to do is select multi-tenancy for your class (or classes). You can keep using all your favorite features, including super-fast queries, cross-references, and replication. The only change? For every operation, specify the tenant key – a user-defined property in your schema. That’s all that changes.Weaviate `v1.20` - coming in July 2023 - changes this once and for all: Native multi-tenancy support that scales to millions of tenants with 10s of thousands of active tenants per node. Yet scale is not the only point tha

## Evaluate Compiled

In [37]:
evaluate(compiled_blog_writer, metric=MetricWrapper)


Average Metric: 22.5 / 5  (450.0): 100%|██████████| 5/5 [06:00<00:00, 72.12s/it]

Average Metric: 22.5 / 5  (450.0%)


,question,blog,MetricWrapper
0,What is a vector database?,"""Understanding Vector Databases: Technical Details, Use Cases, and Security Measures"" A vector database is a type of database that stores data indexed by machine learning...",4.5
1,What is HNSW?,"""Understanding HNSW: Key Features, Comparison with Vamana, and Weaviate's Custom Implementation"" Hierarchical Navigable Small World (HNSW) is a type of vector index that supports querying...",4.5
2,What can I build with Cohere and Weaviate?,"""Getting Started with Cohere and Weaviate: An Introduction to the text2vec-cohere Module and Project Examples"" --- Blog Outline: Introduction to Weaviate, Explanation of Weaviate's GraphQL...",4.5
3,What is a race condition in database systems?,"""Understanding Databases: From Race Conditions and Lock Striping to Persistence and Resiliency"" A race condition is a situation in programming where the sequence of events...",4.5
4,How do inverted indexes work?,"""Inverted Indexes: Functioning, Role in Filtering, and Comparison with Bitmap Indexes"" An inverted index is a type of database index that is primarily used to...",4.5


450.0